In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.13.0+cu117


In [2]:
import torch_geometric
from torch_geometric.datasets import Planetoid

In [17]:
use_cuda_if_available = True

# Load the dataset

In [4]:
dataset = Planetoid(root="tutorial1",name= "Cora")

Processing...
Done!


##### Dataset properties

In [5]:
print(dataset)
print("number of graphs:\t\t",len(dataset))
print("number of classes:\t\t",dataset.num_classes)
print("number of node features:\t",dataset.num_node_features)
print("number of edge features:\t",dataset.num_edge_features)

Cora()
number of graphs:		 1
number of classes:		 7
number of node features:	 1433
number of edge features:	 0


##### Dataset shapes

In [7]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [8]:
print("edge_index:\t\t",dataset.data.edge_index.shape)
print(dataset.data.edge_index)
print("\n")
print("train_mask:\t\t",dataset.data.train_mask.shape)
print(dataset.data.train_mask)
print("\n")
print("x:\t\t",dataset.data.x.shape)
print(dataset.data.x)
print("\n")
print("y:\t\t",dataset.data.y.shape)
print(dataset.data.y)

edge_index:		 torch.Size([2, 10556])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])


train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])


x:		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y:		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [9]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv


In [10]:
data = dataset[0]

In [11]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = SAGEConv(dataset.num_features,
                             dataset.num_classes,
                             aggr="max") # max, mean, add ...)

    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)
    

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() and use_cuda_if_available else 'cpu')

In [19]:
print (device)

cuda


In [20]:
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [21]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [23]:
best_val_acc = test_acc = 0
for epoch in range(1,1000):
    train()
    _, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}'
    
    if epoch % 10 == 0:
        print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 0.7120, Test: 0.7240
Epoch: 020, Val: 0.7140, Test: 0.7270
Epoch: 030, Val: 0.7180, Test: 0.7260
Epoch: 040, Val: 0.7180, Test: 0.7260
Epoch: 050, Val: 0.7180, Test: 0.7260
Epoch: 060, Val: 0.7180, Test: 0.7260
Epoch: 070, Val: 0.7200, Test: 0.7320
Epoch: 080, Val: 0.7220, Test: 0.7340
Epoch: 090, Val: 0.7220, Test: 0.7340
Epoch: 100, Val: 0.7220, Test: 0.7340
Epoch: 110, Val: 0.7220, Test: 0.7340
Epoch: 120, Val: 0.7220, Test: 0.7340
Epoch: 130, Val: 0.7220, Test: 0.7340
Epoch: 140, Val: 0.7220, Test: 0.7340
Epoch: 150, Val: 0.7220, Test: 0.7340
Epoch: 160, Val: 0.7220, Test: 0.7340
Epoch: 170, Val: 0.7220, Test: 0.7340
Epoch: 180, Val: 0.7220, Test: 0.7340
Epoch: 190, Val: 0.7220, Test: 0.7340
Epoch: 200, Val: 0.7220, Test: 0.7340
Epoch: 210, Val: 0.7220, Test: 0.7340
Epoch: 220, Val: 0.7220, Test: 0.7340
Epoch: 230, Val: 0.7220, Test: 0.7340
Epoch: 240, Val: 0.7220, Test: 0.7340
Epoch: 250, Val: 0.7220, Test: 0.7340
Epoch: 260, Val: 0.7220, Test: 0.7340
Epoch: 270, 